## Proces the eval results into csv for mturk eval

In [10]:
import os
import json
import random
import csv

alphabets = ['a', 'b', 'c', 'd', 'e', 'f']
eval_results_dir = "/local1/ponienkung/CtrlGen/coauthor-interface/backend/coauthor_eval_results"
eval_mturkready_dir = "/local1/ponienkung/CtrlGen/coauthor-interface/backend/coauthor_eval_results_mturkready"

# One csv row is a hit, if want multiple examples in one hit, need to add colums
mturk_example_per_hit = 1

# For eval set, every 50 data are from one promt, we get 20 from each prompt
example_per_prompt = 50
use_example_per_prompt = 20

constraints = []

for keyword in [0, 1]:
    for wordlen in [0, 1]:
        for insertion in [0, 1]:
            constraints.append(f"K{keyword}L{wordlen}I{insertion}")




for cont in constraints:
    eval_file_template = "CoauthorEval_{cont}_{model}.json"
    output_file = os.path.join(
        eval_mturkready_dir,
        f"CoauthorEval_{cont}.csv"
    )
    
    # For examples
    models = ["local", "gpt-3.5-turbo", "gpt-4-turbo"]
    model_outputs = {}
    
    for model in models:
        eval_file = os.path.join(eval_results_dir, eval_file_template.format(cont = cont, model = model))
        data = json.load(open(eval_file))
        # Story the dict
        model_outputs[model] = data
    
    session_list = []
    for i, (session, data) in enumerate(model_outputs[models[0]].items()):
        if i % 50 >= 20:
            continue
        prefix = ""
        suffix = ""
        if data['prefix'] != "":
            prefix_trunc = 3 if data['prefix'][-1] == "." else 2
            prefix = ".".join(data['prefix'].split(".")[-prefix_trunc:]).lstrip()
        if data['constraints']['suffix'] != "":
            suffix_trunc = 3 if data['constraints']['suffix'][0] == "." else 2
            suffix = ".".join(data['constraints']['suffix'].split(".")[:suffix_trunc]).rstrip()
        session_list.append({
            "session": session,
            "prefix": prefix,
            "suffix": suffix
        })
    # Also shuffle the session list
    random.shuffle(session_list) 
    # Now for all example in session list, create dict for csv file
    csv_list = []
    for i, session in enumerate(session_list):
        if i % mturk_example_per_hit == 0:
            csv_list.append({})
        csv_rid = i//mturk_example_per_hit
        csv_row = csv_list[csv_rid]
        sid = i%mturk_example_per_hit + 1 # cnt from 0
        # Random shuffle the model list
        random.shuffle(models)
        csv_row[f"idx_{sid}"] = session['session']
        for j, model in enumerate(models):
            csv_row[f"text_{alphabets[j]}_src_{sid}"] = model
        csv_row[f"prefix_{sid}"] = session['prefix'].replace("\n", "<br>")
        for j, model in enumerate(models):
            model_suggestions = model_outputs[model][session['session']]['suggestions']
            csv_row[f"text_{alphabets[j]}_{sid}"] = model_suggestions[0].replace("\n", "<br>") # Use the first one, which has highest prob
        csv_row[f"suffix_{sid}"] = session['suffix'].replace("\n", "<br>")
    
    # Write to csv
    csv_keys = csv_list[0].keys()

    with open(output_file, 'w', newline='') as F:
        dict_writer = csv.DictWriter(F, csv_keys)
        dict_writer.writeheader()
        dict_writer.writerows(csv_list)

### Change the order of the suggestion
Fix the previous bug that the highest probability one appears at the second. This one should only be processed once

In [6]:
# import os
# import json
# from tqdm import tqdm
# eval_results_dir = "/local1/ponienkung/CtrlGen/coauthor-interface/backend/coauthor_eval_results"

# file_list = [f for f in os.listdir(eval_results_dir) if "CoauthorEval" in f]
# for file_name in tqdm(file_list):
#     file_path = os.path.join(eval_results_dir, file_name)
#     try:
#         data = json.load(open(file_path, "r"))
#         for session_id, session_data in data.items():
#             # Switch
#             if "suggestions" in session_data:
#                 tmp = data[session_id]["suggestions"][0]
#                 data[session_id]["suggestions"][0] = data[session_id]["suggestions"][1]
#                 data[session_id]["suggestions"][1] = tmp
#         with open(file_path, "w") as F:
#             json.dump(data, F, indent = 4, ensure_ascii=False)
#     except Exception as e:
#         print(f"Fail at file {file_name} because of exception: {e}")
        


100%|██████████| 24/24 [00:00<00:00, 47.89it/s]


### Check keyword capital issue
Fix the previous bug that some of the keywords are lower cased instead of upper case.

In [27]:
# First check the number of examples that have this issues
import os
import json
from tqdm import tqdm
eval_results_dir = "/local1/ponienkung/CtrlGen/coauthor-interface/backend/coauthor_eval_results"

file_list = [f for f in os.listdir(eval_results_dir) if "CoauthorEval" in f]

def fix_keyword(keyword:str, infix:str) -> str:
    kw_pos = infix.lower().find(keyword)
    original_kw = infix[kw_pos: kw_pos + len(keyword)]
    if keyword == original_kw.lower():
        return original_kw
    else:
        print("Cannot fix keyword --------", keyword, original_kw)
        print(session_data['infix'])
        return ""

file_stats = {}

for file_name in tqdm(file_list):
    file_stats[file_name] = {
        "KeywordError": 0,
        "FixedKeywordError": 0,
        "RemoveSuggestion": 0,
        "RemoveList": [],
    }
    file_path = os.path.join(eval_results_dir, file_name)
    try:
        data = json.load(open(file_path, "r"))
        for session_id, session_data in data.items():
            # Switch            
            for kid, keyword in enumerate(session_data['constraints']['keyword']):
                if not (keyword in session_data['infix']):
                    file_stats[file_name]["KeywordError"] += 1
                    # Check if we can easily switch it back to the correct keyword
                    fixed_keyword = fix_keyword(keyword, session_data['infix'])
                    if fixed_keyword != "":
                        file_stats[file_name]["FixedKeywordError"] += 1
                        if ("suggestions" in session_data) and ("_local.json" in file_name):
                            # Clear suggestions
                            del data[session_id]["suggestions"]
                            file_stats[file_name]["RemoveSuggestion"] += 1
                            file_stats[file_name]["RemoveList"].append(session_id)
                        data[session_id]['constraints']['keyword'][kid] = fixed_keyword
        with open(file_path, "w") as F:
            json.dump(data, F, indent = 4, ensure_ascii=False)
    except Exception as e:
        print(f"Fail at file {file_name} because of exception: {e}")

with open(os.path.join(eval_results_dir, "fix_keyword_stats.json"), "w") as F:
    json.dump(file_stats, F, indent = 4)

 50%|█████     | 12/24 [00:00<00:00, 47.80it/s]

Cannot fix keyword -------- sneak around 
 You sneak  around the side of your house and sidle up to the neighbor's front porch.
Cannot fix keyword -------- sneak around 
 You sneak  around the side of your house and sidle up to the neighbor's front porch.


100%|██████████| 24/24 [00:00<00:00, 40.31it/s]


Cannot fix keyword -------- sneak around 
 You sneak  around the side of your house and sidle up to the neighbor's front porch.


FileNotFoundError: [Errno 2] No such file or directory: '/local1/ponienkung/CtrlGen/coauthor-interface/backend/coauthor_eval_results/fix_keyword_stats.json'